# Inspeccionando el mecanismo de atención de un modelo de lengua

<a target="_blank" href="https://colab.research.google.com/github/jaspock/me/blob/main/docs/materials/assets/misterios/notebooks/alti.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> <a href="http://dlsi.ua.es/~japerez/"><img src="https://img.shields.io/badge/Universitat-d'Alacant-5b7c99" style="margin-left:10px"></a>

Cuaderno preparado por Juan Antonio Pérez en 2025. Este cuaderno permite ver qué palabras de la entrada son más relevantes para el mecanismo de atención de un modelo de lengua a la hora de apostar por la siguiente palabra. Usa para ello una técnica llamada [ALTI-Logit](https://aclanthology.org/2023.acl-long.301/). La mayor parte del código de este cuaderno está tomado del [repositorio](https://github.com/mt-upc/logit-explanations) de los autores de esta técnica.

El entorno de ejecución de este cuaderno ha de tener una GPU. Si usas Google Colab, la puedes conseguir desde el menú *Entorno de ejecución* / *Cambiar tipo de entorno de ejecución*.

## Instalación del código necesario

Los dos primeros bloques solo instalan el código del método ALTI-Logit e importan las librerías necesarias.

In [ ]:
%%capture
!git clone https://github.com/mt-upc/logit-explanations.git
%cd logit-explanations
# avoid installation of specific versions to speed up the process (use at your own risk):
!sed -i -E 's/[><=!~^]+[0-9.]+//g' requirements.txt  
!pip install -r requirements.txt

In [ ]:
import torch
import src.utils_contributions as utils_contributions
from src.contributions import ModelWrapper
from extract_explanations import read_blimp_dataset, track2input_tokens, read_sva_dataset, read_ioi_dataset
import pandas as pd
from lm_saliency import *

device = "cuda" if torch.cuda.is_available() else "cpu"


## Selección del modelo

Aunque el sistema soporta diferentes modelos de lengua de tamaño pequeño, te puedes centrar en los de la familia GPT-2.

Para que funcione la descarga, has de tener una clave secreta de nombre HF_TOKEN con un valor que puedes obtener si te creas una cuenta en Hugging Face. Pregunta a tu profesor antes. Esta clave se añade desde la sección con el icono de la llave a la izquierda de este cuaderno si lo has abierto en Google Colab.

La descarga de los modelos puede llevar unos minutos, especialmente la primera vez que se ejecute el cuaderno en un nuevo entorno de ejecución.

In [ ]:
# Currently tested for:
#  OPT: facebook/opt-125m (bigger models use Post-LN)
#  GPT-2: gpt2 (124M), gpt2-large (774M), gpt2-xl (1.5B)
#  BLOOM: bigscience/bloom-560m bigscience/bloom-1b1
name_path = 'gpt2-xl'
model, tokenizer = utils_contributions.load_model_tokenizer(name_path)
model_wrapped = ModelWrapper(model)

## Contexto y palabras objetivo y sorpresa

Aquí puedes definir el texto (`text`) a utilizar con el modelo de lengua y las dos palabras siguientes que queremos comparar: por un lado, una palabra coherente en el contexto (a la que llamaremos palabra *objetivo* y que se guarda en la variable `target`) y, por otro lado, una que no lo sea (a la que llamaremos palabra *sorpresa* y que se guarda en la variable `foil`).

El texto se segmenta en las diferentes palabras (a veces, unidades más pequeñas) que procesará el modelo. Cuando se imprime la lista de palabras el símbolo `Ġ` representa un espacio en blanco.

In [ ]:
text = "This summer, unlike the previous winter, is being extremely"  # do not end with a space
target = 'hot'
foil = 'cold'  # surprise word

input = text
print('input: ' , input)
print('target: ', target)
print('foil: ', foil)

if 'facebook/opt' in tokenizer.name_or_path:
    # OPT tokenizer adds a BOS token at pos 0
    CORRECT_ID = tokenizer(" "+ target)['input_ids'][1]
    FOIL_ID = tokenizer(" "+ foil)['input_ids'][1]
else:
    CORRECT_ID = tokenizer(" "+ target)['input_ids'][0]
    FOIL_ID = tokenizer(" "+ foil)['input_ids'][0]
if CORRECT_ID == FOIL_ID:
    raise ValueError('Same CORRECT_ID and FOIL_ID')

token = [CORRECT_ID, FOIL_ID]
pt_batch = tokenizer(text, return_tensors="pt").to(device)
input_ids = pt_batch['input_ids']
tokenized_text = tokenizer.convert_ids_to_tokens(pt_batch["input_ids"][0])
print(tokenized_text)
seq_len = len(tokenized_text)

El siguiente bloque muestra las palabras siguientes más frecuentes según el modelo de lengua. Aunque lo valores expresados como *logits* no son directamente probabilidades, sí que se cumple que valores mayores de *logits* suponen probabilidades más altas, aunque no de forma proporcional.

In [ ]:
# Forward-pass
logits, hidden_states, attentions = model_wrapped(pt_batch)

probs = torch.nn.functional.softmax(logits, dim=-1).squeeze()
top_k = 10

token_probs = probs[-1]
sorted_token_probs, sorted_token_values = token_probs.sort(descending=True)
top_k_pred_t_ids = torch.topk(token_probs, k=top_k,dim=-1).indices
top_k_pred_t_tokens = tokenizer.convert_ids_to_tokens(torch.squeeze(top_k_pred_t_ids))
top_k_pred_t_values = torch.topk(token_probs,k=top_k,dim=-1).values

for i in range(top_k):
    print(
        f"Top {i}th token. Logit: {logits[0, -1, sorted_token_values[i]].item():5.3f} Prob: {sorted_token_probs[i].item():6.2%} Token: |{tokenizer.convert_ids_to_tokens(sorted_token_values[i].item())}| String: |{tokenizer.decode(sorted_token_values[i])}|"
    )

predicted_sentence = tokenized_text[1:] + [top_k_pred_t_tokens[0]]

print(f"CORRECT_ID token. Logit: {logits[0, -1, CORRECT_ID].item():5.3f}")
print(f"FOIL_ID token. Logit: {logits[0, -1, FOIL_ID].item():5.3f}")
print('logits diff', logits[0, -1, CORRECT_ID] - logits[0, -1, FOIL_ID])

if model_wrapped.model.config.model_type == 'opt':
    bos_model = True
else:
    bos_model = False

## Visualización de las atenciones

La matriz que se genera acontinuación representa con colores más oscuros aquellas palabras de la entrada a las que el mecanismo de atención da más importancia a la hora de considerar una determinada palabra como siguiente. La matriz se lee normalmente por filas: para la palabra de una determinada fila, los colores de las columnas indican la importancia de las palabras correspondientes.

Puedes observar que cada palabra solo atiende a sí misma y a las palabras anteriores. Esto es así como consecuencia de que los modelos de lengua van calculando las atenciones y predicciones incrementalmente de izquierda a derecha.

In [ ]:
logit_trans_vect_dict, logits_modules, layer_alti_data = model_wrapped.get_logit_contributions(hidden_states, attentions, token)
contributions_mix_alti = utils_contributions.compute_alti(layer_alti_data)

In [ ]:
import seaborn as sns
# Show ALTI input attributions to model outputs
df = pd.DataFrame(np.array(contributions_mix_alti[-1]),columns=tokenized_text,index=tokenized_text)
sns.heatmap(df,cmap="Blues",square=True);

## Análisis contrastivo de las palabras relevantes

El siguiente gráfico muestra la diferencia entre la atención puesta en cada palabra del contexto cuando se predice la palabra objetivo (`target`) y cuando se predice la palabra sorpresa (`foil`). La interpretación de estos valores es la siguiente:

- Un valor positivo significa que esa palabra es importante para predecir la palabra objetivo, pero no para la palabra sorpresa.
- Un valor cercano a cero significa que en ambos casos la palabra tiene una relevancia similar.
- Un valor negativo implica que la palabra es irrelevante en la predicción de la palabra objetivo, pero relevante en la palabra sorpresa.

In [ ]:
# Compute ALTI-Logit
methods_decomp = ['aff_x_j'] # Logits Affine part of layer-wise decomposition
alti_lg_dict = track2input_tokens(logit_trans_vect_dict, methods_decomp, contributions_mix_alti, token)
alti_lg_dict.keys()

In [ ]:
# 'logit_aff_x_j_alti' ALTI-Logit explanation
# 'logit_aff_x_j' Logit explanation
method = 'logit_aff_x_j'

# Contrastive explanation
contrastive_contributions = (alti_lg_dict[method][0] - alti_lg_dict[method][1]).sum(0)

# Add inital logit update by last postion intial embedding (see Eq. 7 paper)
init_logits_diff = (logits_modules['init_logit'][0] - logits_modules['init_logit'][1]).to('cpu')
contrastive_contributions[-1] += init_logits_diff
print(contrastive_contributions.sum())

# Normalization done in Kayo's work
# Divides by the sum of the absolute values (l1) of the explanations vector
norm = np.linalg.norm(contrastive_contributions, ord=1)
contrastive_contributions /= norm
explanations_list = []
explanations_list.append(contrastive_contributions)
# Yin and Neubig visualization (https://github.com/kayoyin/interpret-lm)
# visualize(np.array(contrastive_contributions), tokenizer, [pt_batch["input_ids"][0]], print_text=True, normalize=False)
# Barplot visualization
utils_contributions.plot_histogram(contrastive_contributions,tokenized_text)